In [41]:
import numpy as np
from keras.layers import Input, Conv2D, Flatten, Dense, LeakyReLU, Conv2DTranspose, Reshape, Activation
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.datasets import mnist

In [42]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

Create encoder model

In [16]:
encoder_input = Input((28, 28, 1))

e = Conv2D(
    filters=32,
    kernel_size=3,
    strides=1,
    padding='same'
    )(encoder_input)
e = LeakyReLU()(e)

e = Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    padding='same'
    )(e)
e = LeakyReLU()(e)

e = Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    padding='same'
    )(e)
e = LeakyReLU()(e)

e = Conv2D(
    filters=64,
    kernel_size=3,
    strides=1,
    padding='same'
    )(e)
e = LeakyReLU()(e)

shape_before_flattening = K.int_shape(e)[1:]

e = Flatten()(e)
encoder_output = Dense(2)(e)

encoder_model = Model(encoder_input, encoder_output)

encoder_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 7, 7, 64)          0   

Create decoder model

In [29]:
decoder_input = Input((2,))

d = Dense(np.prod(shape_before_flattening))(decoder_input)
d = Reshape(shape_before_flattening)(d)

d = Conv2DTranspose(
    filters=64,
    kernel_size=3,
    strides=1,
    padding='same'
    )(d)
d = LeakyReLU()(d)

d = Conv2DTranspose(
    filters=64,
    kernel_size=3,
    strides=2,
    padding='same'
    )(d)
d = LeakyReLU()(d)

d = Conv2DTranspose(
    filters=32,
    kernel_size=3,
    strides=2,
    padding='same'
    )(d)
d = LeakyReLU()(d)

d = Conv2DTranspose(
    filters=1,
    kernel_size=3,
    strides=1,
    padding='same'
    )(d)
d = Activation('sigmoid')(d)

decoder_output = d

decoder_model = Model(decoder_input, decoder_output)

decoder_model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 3136)              9408      
_________________________________________________________________
reshape_7 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_9 (Conv2DTr (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 14, 14, 64)        0   

Full autoencoder

In [31]:
ae_input = encoder_input
ae_output = decoder_model(encoder_output)

ae_model = Model(ae_input, ae_output)

Training

In [30]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [37]:
def r_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis = [1,2,3])

In [39]:
optimizer = Adam(lr=LEARNING_RATE)
ae_model.compile(optimizer=optimizer, loss=r_loss)

In [36]:
ae_model.fit(     
    x_train,
    x_train,
    batch_size = batch_size,
    shuffle = True,
    epochs = epochs,
    initial_epoch = initial_epoch,
    callbacks = callbacks_list
    )

In [ ]:
AE.train(     
    x_train[:1000],
    batch_size = BATCH_SIZE,
    epochs = 200,
    run_folder = RUN_FOLDER,
    initial_epoch = INITIAL_EPOCH
)